# Load data

In [1]:
import pandas as pd
df=pd.read_parquet('data/AirData_raw.parquet')
df.head(3)

,Estacion,Medida,Tecnica,Year,Month,Day,Value,Date
0,Pza.deEspaña,Dióxido_de_Azufre,Fluorescencia_ultravioleta,2003,1,1,6.0,2003-01-01
1,Pza.deEspaña,Dióxido_de_Azufre,Fluorescencia_ultravioleta,2003,2,1,19.0,2003-02-01
2,Pza.deEspaña,Dióxido_de_Azufre,Fluorescencia_ultravioleta,2003,3,1,15.0,2003-03-01


In [2]:
df['Medida']=[i.replace('_',' ') for i in df['Medida']]
df['Tecnica']=[i.replace('_',' ') for i in df['Tecnica']]

# Detect outliers and label them on the main table

In [3]:
# Compute a simple thr
th_df=(df[['Medida','Estacion','Value']].groupby(['Estacion','Medida']).std()*5).reset_index()
th_df=th_df.rename(columns={'Value':'Th'})
# Add thresholds to the main table and label anomalies
df=df.merge(th_df, left_on=['Medida','Estacion'], right_on=['Medida','Estacion'])
df['Anomaly']=df['Value']>=df['Th']

In [4]:
# Rename columns and store
df.columns=['Location','Contaminant','Technique','Year', 'Month', 'Day', 'Value','Date', 'Th', 'Outlier']
df.to_parquet('data/AirData.parquet')

# Store monthly totals

In [5]:
# Count monthly anomalies per location and contaminant 
df_anomalies=df[['Location','Contaminant','Year','Month','Outlier']].groupby(['Location','Contaminant','Year','Month']).sum().reset_index()
# Rename columns
df_anomalies.columns=['Location','Contaminant','Year','Month','Anomalies']
# Save to file, we can make some stats later...
df_anomalies.to_parquet('data/Anomalies_count.parquet')